In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from textblob import TextBlob
import seaborn as sns

from sklearn.model_selection import train_test_split
import tensorflow as tf

In [3]:
df=pd.read_csv('dataset.csv')
#dataset = pd.read_csv('/kaggle/input/200k-short-texts-for-humor-detection/dataset.csv')
df



,text,humor
0,"Joe biden rules out 2020 bid: 'guys, i'm not r...",False
1,Watch: darvish gave hitter whiplash with slow ...,False
2,What do you call a turtle without its shell? d...,True
3,5 reasons the 2016 election feels so personal,False
4,"Pasco police shot mexican migrant from behind,...",False
...,...,...
199995,Conor maynard seamlessly fits old-school r&b h...,False
199996,How to you make holy water? you boil the hell ...,True
199997,How many optometrists does it take to screw in...,True
199998,Mcdonald's will officially kick off all-day br...,False


In [4]:
lengths=[]

for i in range(len(df.text)):
    lengths.append(len(df.text[i].split(' ')))

word_len=[]
for i in range(len(df)):
    word_len.append(len(df.text[i].split(' ')))
print("Maximum words in a sentence:",max(word_len))
print("Average words:",round(np.mean(word_len)))

text=df.text.to_list()

max_words=100000
tok=tf.keras.preprocessing.text.Tokenizer(
    num_words=max_words,
    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
    lower=True,
    split=' ',
    char_level=False,
    oov_token=None,
    document_count=0)

Maximum words in a sentence: 62
Average words: 12


In [5]:
tok.fit_on_texts(text)

In [6]:
train_seq=tok.texts_to_sequences(text)

In [7]:
maxlen = max([len(x) for x in train_seq])

maxlen

25

In [8]:
text_padded = tf.keras.preprocessing.sequence.pad_sequences(
    train_seq,
    maxlen=maxlen,
    dtype='int32',
    padding='post',
    truncating='post',
    value=0.0
)
text_padded.shape

(200000, 25)

In [9]:
df.humor=(df.humor==True)*1
df.humor.value_counts()

0    100000
1    100000
Name: humor, dtype: int64

In [10]:
X_train, X_test, Y_train, Y_test= train_test_split(text_padded, df.humor,
                                                  test_size=0.2, random_state=1729)

In [11]:
X_train.shape, Y_train.shape

((160000, 25), (160000,))

In [12]:
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import LSTM, Dense, Bidirectional,InputLayer,Embedding, Flatten,Input,Dropout

In [13]:
model=tf.keras.Sequential()

model.add(Input(shape=[maxlen]))
model.add(Embedding(max_words,128,input_length=maxlen))    

model.add(LSTM(100, return_sequences=True))
model.add(Dropout(0.5))

#model.add(LSTM(200, return_sequences=True))
#model.add(Dropout(0.5))

model.add(tf.keras.layers.Dense(1,activation='sigmoid')) 

In [14]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [15]:
model.fit(X_train,Y_train,batch_size=256,validation_data=(X_test, Y_test),epochs=20)

Epoch 1/20
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
625/625 [==============================] - ETA: 0s - loss: 0.2305 - accuracy: 0.9092WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x00000118BB05F2E0> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the

In [16]:
X_test

array([[   17,    12,    75, ...,     0,     0,     0],
       [    9,    12,     4, ...,     0,     0,     0],
       [  903, 12862,   676, ...,     0,     0,     0],
       ...,
       [ 8000,   898,  1050, ...,     0,     0,     0],
       [    4,    91,  2492, ...,     0,     0,     0],
       [ 2177,   357,   420, ...,     0,     0,     0]])

In [17]:
Y_pred=model.predict(X_test)

print("All Probabilities:\n",Y_pred[0])
print("\nCorrect:", Y_pred[0][-1])

In [ ]:
Y_preds=[]
for i in range(len(Y_pred)):
    Y_preds.append(Y_pred[i][-1])
Y_preds=np.array(Y_preds)

In [ ]:
Y_preds=(Y_preds>0.5)*1

In [ ]:
Y_test=np.array(Y_test).reshape(-1,1)

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
import seaborn as sns, matplotlib.pyplot as plt


plt.figure(figsize=(10,6))
sns.heatmap(confusion_matrix(Y_preds,Y_test),annot=True,fmt='d',cmap='Blues')
plt.title("Confusion Matrix, 0 signifies not sarcastic, 1 signifies sarcastic")
plt.show()



print("Classification report:", classification_report(Y_preds,Y_test))